In [ ]:
from asbe.base import *
from asbe.models import *
from asbe.estimators import *
import pandas as pd
import numpy as np
from openbt.openbt import OPENBT
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from copy import deepcopy
import econml

/opt/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/AMLab-Amsterdam/CEVAE/master/datasets/IHDP/csv/ihdp_npci_1.csv",
    names = ["treatment", "y_factual", "y_cfactual", "mu0", "mu1"] + [f'x{x}' for x in range(25)])

In [ ]:
X = df.loc[:,"x0":].to_numpy()
t = df["treatment"].to_numpy()
#t = np.zeros_like(t)
y = df["y_factual"].to_numpy()
y1 = np.where(df["treatment"] == 1,
               df['y_factual'],
               df['y_cfactual'])
y0 = np.where(df["treatment"] == 0,
               df['y_factual'],
               df['y_cfactual'])
ite = np.where(df["treatment"] == 1,
               df['y_factual'] - df["y_cfactual"],
               df['y_cfactual'] - df["y_factual"])

In [ ]:
X_train, X_test, t_train, t_test, y_train, y_test, ite_train, ite_test, y1_train, y1_test, y0_train, y0_test = train_test_split(
    X, t, y, ite, y1, y0,  test_size=0.9, random_state=1005)
ds = {"X_training": X_train,
     "y_training": y_train,
     "t_training": t_train,
     "X_pool": deepcopy(X_test), 
     "y_pool": deepcopy(y_test),
     "t_pool": deepcopy(t_test),
     "y1_pool": y1_test,
     "y0_pool":y0_test,
     "X_test": X_test,
     "y_test": y_test,
      "t_test": t_test,
      "ite_test": ite_test
     }
asl = BaseActiveLearner(estimator = BaseITEEstimator(model = RandomForestRegressor(),
                                         two_model=False,ps_model=None),
                        acquisition_function=RandomAcquisitionFunction(),
                        assignment_function=MajorityAssignmentFunction(),
                        stopping_function = None,
                        dataset=ds)

In [ ]:
asl.fit()
X_new, query_idx = asl.query(no_query=120)
asl.teach(query_idx)
preds = asl.predict(asl.dataset["X_test"])
asl.score()

[3.59742915 3.16862861 3.14697116 3.17384683 3.25810398 2.964646
 3.89935253 3.56663741 1.48758712 3.86585774 1.60327876 4.50690703
 3.68481651 1.70743395 2.62708806 4.03307811 0.66017412 3.1987421
 3.43168311 0.91882498 4.30285053 3.58729401 3.40733028 3.77401418
 4.14909976 2.23346215 4.40683687 3.86772822 1.75282784 3.06400544
 4.45583143 3.46696957 4.87554688 2.83682842 1.75456744 4.57731933
 2.52758491 3.63968939 2.19171436 4.06442213 4.02012336 2.1001374
 5.06544685 3.10472589 4.00783396 3.97573757 4.58127209 2.86521353
 2.9596649  4.07109203 5.10350592 4.41841313 2.21861243 3.63404224
 4.1710471  2.96595387 3.6794684  3.81504469 2.63768511 4.3403089
 2.72542292 0.92663386 3.09917078 0.61005313 4.18845563 3.36730817
 2.65694534 3.10077671 3.05508198 2.60418499 3.82394059 2.52972118
 2.95552328 1.74917622 2.1526982  1.87392963 3.61116822 3.27873219
 1.72937344 2.54490522 3.1209124  1.76690179 1.06111346 4.03493186
 2.72239143 3.89480998 2.48019905 0.86758009 4.01914485 4.22828959


In [ ]:
bite = OPENBTITEEstimator(model=OPENBT(model="bart"),
                          two_model = False,
                          ps_model  = None,
                          dataset   = ds)
bite.fit()
preds_obt = bite.predict(X = ds["X_test"])
print(preds_obt.shape)

Overwriting k to agree with the model's default
Overwriting overallnu to agree with the model's default
Overwriting ntree to agree with the model's default
Overwriting ntreeh to agree with the model's default
Overwriting overallsd to agree with the model's default
Writing config file and data
/var/folders/44/gtm_t6x110jg6b13p4rbwkfh0000gn/T/openbtpy_w75piihg
3+ x variables
Running model...
(673, 1000)


In [ ]:
bite = CausalForestEstimator(
                        two_model = None,
                        ps_model  = None,
                        dataset   = ds)
bite.fit()
preds_cf = bite.predict(X = ds["X_test"])

In [ ]:
cite = CEVAEEstimator(dataset=ds,two_model=None)
cite.fit()
preds_ce = cite.predict(X=ds["X_test"])

INFO 	 Training with 1 minibatches per epoch
DEBUG 	 step     0 loss = 51.6576
INFO 	 Evaluating 7 minibatches
DEBUG 	 batch ate = 1.4456
DEBUG 	 batch ate = 1.55399
DEBUG 	 batch ate = 1.53096
DEBUG 	 batch ate = 1.66902
DEBUG 	 batch ate = 1.54256
DEBUG 	 batch ate = 1.72028
DEBUG 	 batch ate = 1.73469
